### Project Scaffolding

In [1]:
# --- Environment & paths ---
import os, sys, warnings, math, json, textwrap
from pathlib import Path
import numpy as np
import pandas as pd

# Reproducibility
np.random.seed(42)

# Base project folder (adjust if your notebook sits elsewhere)
PROJ = Path.cwd()              # e.g., .../cvm
DATA = PROJ                    # data files live here per your screenshot
ART = PROJ / "artifacts"
FIG = ART / "figures"
CSV = ART / "csv"
for d in [ART, FIG, CSV]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Project: {PROJ}")
print(f"Data files present:", [p.name for p in DATA.glob("*.xlsx")])

Project: /Users/atheeshkrishnan/AK/DEV/econometric-forecasting/cvm
Data files present: ['CPI-Data.xlsx', 'CMO-Historical-Data-Monthly.xlsx']


### Data Ingestion

In [2]:
# ---------- Helpers ----------
def coerce_datetime(col):
    """Coerce a column to pandas Period(M) if monthly, else DatetimeIndex."""
    s = pd.to_datetime(col, errors="coerce")
    # treat monthly series as MonthEnd dates
    s = s.dt.to_period("M").dt.to_timestamp("M")
    return s

def read_cmo_xlsx(path: Path, sheet=None, header=None):
    """
    Robust loader for World Bank CMO Monthly workbook.
    Returns a tidy DataFrame with Date index and commodity columns (float).
    - Set `sheet` and `header` if you know them; else we auto-detect.
    """
    # Try a direct read; if it fails, try first sheet and search for 'Date'
    try:
        df0 = pd.read_excel(path, sheet_name=sheet, header=header)
    except Exception:
        df0 = pd.read_excel(path, sheet_name=0, header=None)

    # Heuristic: locate the row whose first cell matches 'Date' (case-insensitive)
    if header is None:
        hdr_row = None
        for i in range(min(30, len(df0))):
            v = str(df0.iloc[i, 0]).strip().lower()
            if v in ("date", "month", "time"):
                hdr_row = i
                break
        if hdr_row is not None:
            df0 = pd.read_excel(path, sheet_name=sheet or 0, header=hdr_row)

    # Drop fully-empty cols/rows
    df0 = df0.dropna(axis=0, how="all").dropna(axis=1, how="all")

    # Expect first column to be date-like
    date_col = df0.columns[0]
    df0[date_col] = coerce_datetime(df0[date_col])
    df0 = df0.rename(columns={date_col: "Date"}).set_index("Date")

    # Keep only numeric columns (commodity price series)
    df_num = df0.select_dtypes(include=[np.number]).copy()
    # Forward-fill occasional gaps (typical in monthly sheets)
    df_num = df_num.sort_index().ffill()

    return df_num

def read_cpi_xlsx(path: Path, sheet=None, header=None):
    """
    Flexible CPI loader. Accepts common BLS formats:
    - Columns like ['DATE','Value'] or ['Year','Period','Value'] or custom ['Date','CPI'].
    Returns a monthly Date index and a single 'CPI' column as float.
    """
    df = pd.read_excel(path, sheet_name=sheet, header=header)
    df = df.dropna(how="all", axis=1).dropna(how="all", axis=0)

    # Try to find a date column
    candidates_date = [c for c in df.columns if str(c).lower() in ("date","month","period","time","observation_date")]
    if not candidates_date:
        # try common BLS wide → long conversions
        # if year/period provided, build a date
        if {"year","period","value"}.issubset({str(c).lower() for c in df.columns}):
            cmap = {c:str(c).lower() for c in df.columns}
            y = df[[c for c in df.columns if str(c).lower()=="year"][0]]
            p = df[[c for c in df.columns if str(c).lower()=="period"][0]].astype(str).str.replace("M","", regex=False)
            v = df[[c for c in df.columns if str(c).lower()=="value"][0]]
            dt = pd.to_datetime(y.astype(str) + "-" + p.str.zfill(2) + "-01") + pd.offsets.MonthEnd(0)
            out = pd.DataFrame({"Date": dt, "CPI": pd.to_numeric(v, errors="coerce")})
            return out.dropna().set_index("Date").sort_index()
        else:
            raise ValueError("Could not find a recognizable date structure in CPI file.")

    date_col = candidates_date[0]
    # Try to find value column
    candidates_val = [c for c in df.columns if str(c).lower() in ("value","cpi","cpiall","cpisa","index","obs_value")]
    if not candidates_val:
        # if exactly two columns, assume the non-date is value
        if df.shape[1] == 2:
            candidates_val = [c for c in df.columns if c != date_col]
        else:
            raise ValueError("Could not find a CPI value column. Rename one column to 'Value' or 'CPI'.")

    val_col = candidates_val[0]
    out = pd.DataFrame({
        "Date": coerce_datetime(df[date_col]),
        "CPI": pd.to_numeric(df[val_col], errors="coerce")
    }).dropna()
    return out.set_index("Date").sort_index()


In [3]:
# ---------- Load data ----------
CMO_PATH = DATA / "CMO-Historical-Data-Monthly.xlsx"
CPI_PATH = DATA / "CPI-Data.xlsx"

prices_nominal = read_cmo_xlsx(CMO_PATH)
cpi = read_cpi_xlsx(CPI_PATH)

print("CMO shape:", prices_nominal.shape)
print("CPI shape:", cpi.shape)
print("CMO date range:", prices_nominal.index.min(), "→", prices_nominal.index.max())
print("CPI  date range:", cpi.index.min(), "→", cpi.index.max())
print("First 10 commodity columns:", list(prices_nominal.columns[:10]))

AttributeError: 'dict' object has no attribute 'iloc'